# Exploring Individual Fits

This notebook looks at fit results for individual light-curves.


In [ ]:
import sys
from copy import deepcopy
from pathlib import Path

import pandas as pd
import sncosmo
from IPython.display import clear_output
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
from analysis_pipeline.data_access import csp, des, sdss
from analysis_pipeline import SN91bgSource


In [ ]:
salt2_20 = sncosmo.Model(source='salt2', version='2.0')
salt2_24 = sncosmo.Model(source='salt2', version='2.4')
sn_91bg = sncosmo.Model(source=SN91bgSource())


## Individual fits

We start by looking at fits individually

In [ ]:
fit_results_dir = Path('../fit_results')

def set_model(cid, survey, model, num_params, bands):
    """Set model parameters using fit results
    
    Args:
        cid        (str): Object id
        survey     (str): Survey name (eg. csp)
        model    (Model): An SNCosmo model
        num_params (int): The number of fitted parameters (4 or 5)
        bands      (str): 'all', 'blue', or 'red'
        
    Returns:
        A copy of ``model`` with set parameter values
    """    

    model_name = f'{model.source.name}_{model.source.version}'
    fname = f'{survey}_{num_params}_{model_name}_{bands}.csv'
    table_path = fit_results_dir / survey / fname

    results_df = pd.read_csv(table_path)
    results_for_cid = results_df[results_df['cid'] == cid]
    
    out_model = deepcopy(model)
    out_model.set(
        z=results_for_cid['z'],
        t0=results_for_cid['t0'],
        x0=results_for_cid['x0'],
        x1=results_for_cid['x1'],
        c=results_for_cid['c']
    )
    
    return out_model


def plot_fit(cid, survey, model, num_params, bands):
    """Plot a fitted light curve
    
    Args:
        cid        (str): Object id
        survey     (str): Survey name (eg. csp)
        model    (Model): An SNCosmo model
        num_params (int): The number of fitted parameters (4 or 5)
        bands      (str): 'all', 'blue', or 'red'
    """

    data_module = {'csp': csp, 'des': des, 'sdss': sdss}[survey] 
    data = data_module.get_input_for_id(cid)
    model = set_model(cid, survey, model, num_params, bands)
    sncosmo.plot_lc(data, model)
    plt.show()


In [ ]:
plot_fit('2005kc', 'csp', salt2_24, 4, 'all')

## Iteritive Fits

Next we iterate over targets and look at look at fits in all, blue, and red bands.

In [ ]:
def iter_fit_results(survey, model, num_params):
    """Iteratively plot fit results for all, blue, and red bands
    
    Pause for input after aver target
    
    Args:
        survey     (str): Survey name (eg. csp)
        model    (Model): An SNCosmo model
        num_params (int): The number of fitted parameters (4 or 5)
    """
    
    data_module = {'csp': csp, 'des': des, 'sdss': sdss}[survey] 
    for cid in data_module.get_target_ids():
        clear_output()
        plot_fit(cid, survey, model, num_params, 'all')
        plot_fit(cid, survey, model, num_params, 'blue')
        plot_fit(cid, survey, model, num_params, 'red')
        
        i = input()
        if i != '':
            return


In [ ]:
iter_fit_results('csp', salt2_24, 4)